# General Preparations

In [7]:
import sys
sys.path.insert(0, '/yunity/arusty/PF-GAP')

#Import libraries, MDS from SKlearn, and all the custom files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from Pipeline.functions import *


#Data
from aeon.datasets import load_gunpoint
from aeon.datasets import load_basic_motions


#Functions
from QGAP.qgap import QGAP
from Redcomets.Redcomets import REDCOMETS
from RFGAP_Rocket.RFGAP_Rocket import RFGAP_Rocket
from RDST.rdst import RDST_GAP

In [4]:
# Load Data
XTRAIN, YTRAIN = load_gunpoint(split="TRAIN")
XTEST, YTEST = load_gunpoint(split="TEST")

# XTRAIN, YTRAIN = load_basic_motions(split="TRAIN")
# XTEST, YTEST = load_basic_motions(split="TEST")

#Generate random static variables to test algorithm
XSTATIC = np.random.rand(XTRAIN.shape[0], 45)
XSTATIC_TEST = np.random.rand(XTEST.shape[0], 45)


In [8]:
#Quant
qgap = QGAP(matrix_type="dense")
quant_prox = data_to_proximities(qgap, XTRAIN, YTRAIN, XTEST, XSTATIC,XSTATIC_TEST)

#Redcomets
redcomets = REDCOMETS(variant = 3, static = XSTATIC)
reddcomets_prox = data_to_proximities(redcomets, XTRAIN, YTRAIN, XTEST, None , None)

# Rocket
rf_rocket = RFGAP_Rocket(prediction_type = "classification", rocket = "Multi",
                         n_kernels=512) # Rocket Kwargs
rocket_prox = data_to_proximities(rf_rocket, XTRAIN, YTRAIN, XTEST, XSTATIC , XSTATIC_TEST)

# RDST
rdst = RDST_GAP(save_transformed_data = True)
rdst_prox = data_to_proximities(rdst, XTRAIN, YTRAIN, None, XSTATIC , None)


In [13]:
type(rocket_prox), type(rdst_prox), type(quant_prox), type(reddcomets_prox)

(numpy.matrix, numpy.matrix, numpy.ndarray, numpy.ndarray)

# Building MDS Veiwpoints

In [10]:
# Initialize MDS
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)

def build_mds(prox):
    symetric = (prox + prox.T) / 2
    mds_prox = mds.fit_transform(symetric)
    return mds_prox

# Build MDS for each method
mds_quant = build_mds(quant_prox)
mds_redcomets = build_mds(reddcomets_prox)
mds_rocket = build_mds(rocket_prox)
mds_rdst = build_mds(rdst_prox)

# Plot the results
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.scatter(mds_quant[:, 0], mds_quant[:, 1], c=YTEST, cmap='viridis', marker='o', s=10)
plt.title('MDS - QGAP')
plt.subplot(2, 2, 2)
plt.scatter(mds_redcomets[:, 0], mds_redcomets[:, 1], c=YTEST, cmap='viridis', marker='o', s=10)
plt.title('MDS - REDCOMETS')
plt.subplot(2, 2, 3)
plt.scatter(mds_rocket[:, 0], mds_rocket[:, 1], c=YTEST, cmap='viridis', marker='o', s=10)
plt.title('MDS - Rocket')
plt.subplot(2, 2, 4)
plt.scatter(mds_rdst[:, 0], mds_rdst[:, 1], c=YTEST, cmap='viridis', marker='o', s=10)
plt.title('MDS - RDST')
plt.tight_layout()


TypeError: np.matrix is not supported. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html